In [1]:
import pandas as pd
import joblib
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [11]:
# Iniciar o servidor de rastreamento do MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:9080")

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\MICRODADOS_ENEM_2023_tratados.pkl')

Label Encoding: apenas transforma variáveis categóricas em numéricas, não há significado numérico

- Processamento necessário para modelos aplicados: decision tree

In [3]:
# Aplicar o LabelEncoder para converter as colunas categóricas em numéricas
label_encoder = LabelEncoder()
categorical_columns = df_enem.select_dtypes(include=['category']).columns

for col in categorical_columns:
    df_enem[col] = label_encoder.fit_transform(df_enem[col])

In [4]:
df_enem.head()

,BIN_Q001_DUMMY_H,BIN_Q002_DUMMY_H,BIN_Q018,BIN_Q020,BIN_Q021,BIN_Q023,BIN_Q025,CAT_COR_RACA,CAT_CO_MUNICIPIO_ESC,CAT_CO_UF_ESC,...,NUM_Q011,NUM_Q012,NUM_Q013,NUM_Q014,NUM_Q015,NUM_Q016,NUM_Q017,NUM_Q019,NUM_Q022,NUM_Q024
0,False,False,False,False,False,False,True,3,941,9,...,0,1,0,0,0,0,0,1,3,0
1,False,False,False,False,False,False,False,3,1031,9,...,0,1,0,0,0,0,0,1,1,0
2,False,False,False,False,False,False,True,3,4232,20,...,0,1,0,1,0,0,0,1,1,1
3,False,False,False,False,False,True,True,1,169,4,...,0,1,0,0,0,0,0,1,3,0
4,False,False,False,False,False,False,True,1,3770,19,...,0,1,0,1,0,0,0,1,1,0


In [5]:
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

Modelo de Árvore simples como base

In [7]:
# Treinar modelos de árvore de decisão para regressão
modelo_arvore_decisao = DecisionTreeRegressor(random_state=42,
                                               max_depth=5, 
                                               min_samples_split=10, 
                                               min_samples_leaf=5,
                                               splitter='best')

modelo_arvore_decisao.fit(X_train, y_train['NUM_NOTA_CH'])

DecisionTreeRegressor(max_depth=5, min_samples_leaf=5, min_samples_split=10,
                      random_state=42)

In [18]:
# Criar experimento no MLflow
mlflow.set_experiment('Notas CH ENEM 2023')

<Experiment: artifact_location='mlflow-artifacts:/299918284299748162', creation_time=1746134789450, experiment_id='299918284299748162', last_update_time=1746134789450, lifecycle_stage='active', name='Notas CH ENEM 2023', tags={}>

In [8]:
# Avaliação do modelo
y_pred_arvore_decisao = modelo_arvore_decisao.predict(X_test)

In [20]:
# Iniciar o rastreamento do MLflow
with mlflow.start_run() as run:

    # Registrar os parâmetros
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("min_samples_split", 10)
    mlflow.log_param("min_samples_leaf", 5)
    mlflow.log_param("splitter", "best")
    mlflow.log_param("random_state", 42)

    # Registrar as métricas
    r2 = r2_score(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)
    mae = mean_absolute_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)
    rmse = root_mean_squared_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)
    
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("rmse", rmse)
    
    # Definir uma TAG para o modelo
    mlflow.set_tag("model_type", "Decision Tree Regressor")

    # Inferir assinatura do modelo
    signature = mlflow.models.infer_signature(X_train, y_train['NUM_NOTA_CH'])

    # Registrar o modelo
    mlflow.sklearn.log_model(sk_model=modelo_arvore_decisao,
                             artifact_path="modelo_arvore_decisao_base",
                             signature=signature,
                             registered_model_name="modelo_arvore_decisao_base")


Registered model 'modelo_arvore_decisao_base' already exists. Creating a new version of this model...
2025/05/02 10:47:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_arvore_decisao_base, version 6


🏃 View run sneaky-moth-594 at: http://127.0.0.1:9080/#/experiments/299918284299748162/runs/3aaa5d8dc645487aac94933ef1537aab
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/299918284299748162


Created version '6' of model 'modelo_arvore_decisao_base'.


In [21]:
# Função para calcular as métricas
def avaliar_modelo(y_true, y_pred, grupo):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"MAE ({grupo}): {mae:.2f}")
    print(f"RMSE ({grupo}): {rmse:.2f}")
    print(f"R2 ({grupo}): {r2:.2f}")

In [22]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_arvore_decisao.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao, "teste")

MAE (treino): 57.71
RMSE (treino): 72.78
R2 (treino): 0.26
MAE (teste): 57.73
RMSE (teste): 72.85
R2 (teste): 0.26


In [ ]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_arvore_decisao, 'Modelos\modelo_arvore_decisao_base.pkl')

<class 'sklearn.tree._classes.DecisionTreeRegressor'>


['Projeto\\Modelos\\modelo_arvore_decisao_base.pkl']

#### Busca melhor ccp_alpha

##### ccp_alpha:

- Controla a poda da árvore, removendo nós com impacto mínimo no erro.
- Valores maiores resultam em árvores mais simples e generalizáveis.
- O método cost_complexity_pruning_path é usado para calcular os valores possíveis de ccp_alpha.

In [24]:
path = modelo_arvore_decisao.cost_complexity_pruning_path(X_train, y_train['NUM_NOTA_CH'])  # CCP Path na base de treino
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [25]:
ccp_alphas = pd.Series(ccp_alphas).unique()
ccp_alphas

array([0.00000000e+00, 7.74148007e-02, 1.99599818e-01, 4.46745517e-01,
       7.36390767e-01, 9.27910009e-01, 1.17023237e+00, 1.42593009e+00,
       1.69521130e+00, 1.74845783e+00, 2.85736260e+00, 3.21881372e+00,
       4.23505379e+00, 4.51365740e+00, 5.10712784e+00, 5.11296416e+00,
       5.92589067e+00, 7.48206114e+00, 1.05610267e+01, 1.52115718e+01,
       1.70171253e+01, 2.01020180e+01, 2.43590060e+01, 2.71705968e+01,
       3.70206004e+01, 4.50195920e+01, 6.40821984e+01, 8.79949916e+01,
       9.42295521e+01, 9.53932897e+01, 2.92455485e+02, 9.88213558e+02])

In [26]:
# order the values of ccp_alphas
ccp_alphas = sorted(ccp_alphas, reverse=False)
ccp_alphas

[np.float64(0.0),
 np.float64(0.07741480065303596),
 np.float64(0.19959981774929147),
 np.float64(0.4467455172613839),
 np.float64(0.7363907671159637),
 np.float64(0.9279100091976886),
 np.float64(1.1702323684151565),
 np.float64(1.4259300936420658),
 np.float64(1.6952113025986364),
 np.float64(1.7484578348178843),
 np.float64(2.8573626041251288),
 np.float64(3.218813716916088),
 np.float64(4.235053793142015),
 np.float64(4.513657395537052),
 np.float64(5.107127839418524),
 np.float64(5.112964159383466),
 np.float64(5.9258906722836855),
 np.float64(7.4820611412902736),
 np.float64(10.56102665198),
 np.float64(15.211571846605807),
 np.float64(17.017125302427985),
 np.float64(20.10201796487013),
 np.float64(24.359006011949077),
 np.float64(27.170596834820344),
 np.float64(37.02060041316258),
 np.float64(45.019591960726075),
 np.float64(64.08219840231914),
 np.float64(87.99499164361123),
 np.float64(94.22955214772401),
 np.float64(95.39328968923382),
 np.float64(292.4554846315623),
 np.fl

In [27]:
len(ccp_alphas)  # Número de valores únicos de alpha

32

In [28]:
# metricas_calculadas = []
# contador = 0

# for ccp_alpha in ccp_alphas:
#     print(f"Iteração {contador} de {len(ccp_alphas)}")
#     contador += 1

#     # Treinar o modelo de árvore de decisão com o valor de ccp_alpha atual
#     modelo_arvore_decisao = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
#     modelo_arvore_decisao.fit(X_train, y_train['NUM_NOTA_CH'])
#     y_pred_arvore_decisao = modelo_arvore_decisao.predict(X_test)

#     # Avaliação do modelo
#     mae_arvore_decisao_teste = mean_absolute_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)
#     print(f"MAE (teste): {mae_arvore_decisao_teste:.2f}")
#     # rmse_arvore_decisao_teste = root_mean_squared_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)
#     # r2_arvore_decisao_teste = r2_score(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao)

#     # Salvar métricas
#     metricas_calculadas.append({
#         'ccp_alpha': ccp_alpha,
#         'MAE': mae_arvore_decisao_teste,
#         # 'RMSE': rmse_arvore_decisao_teste,
#         # 'R2': r2_arvore_decisao_teste
#     })

#     # MAE atual maior que os dois anteriores parar processamento
#     if contador > 2 and metricas_calculadas[-1]['MAE'] > metricas_calculadas[-2]['MAE'] and metricas_calculadas[-2]['MAE'] > metricas_calculadas[-3]['MAE']:
#         print(f"Parando o loop no alpha {ccp_alpha} com MAE {mae_arvore_decisao_teste:.2f}")
#         break

In [29]:
# # Melhor valor de alpha para MAE mínimo
# melhor_mae = min(metricas_calculadas, key=lambda x: x['MAE'])

# # Melhor valor de alpha para RMSE mínimo
# # melhor_rmse = min(metricas_calculadas, key=lambda x: x['RMSE'])

# # Melhor valor de alpha para R2 máximo
# # melhor_r2 = max(metricas_calculadas, key=lambda x: x['R2'])

# # Melhores valores de alpha
# print(f"Melhor alpha: {melhor_mae['ccp_alpha']:.4f}, com MAE: {melhor_mae['MAE']:.2f}")
# # print(f"Melhor alpha: {melhor_rmse['ccp_alpha']:.4f}, com RMSE: {melhor_rmse['RMSE']:.2f}")
# # print(f"Melhor alpha: {melhor_r2['ccp_alpha']:.4f}, com R2: {melhor_r2['R2']:.2f}")

Melhor CCP Alpha para menor MAE: 0.3436

In [30]:
ccp_alpha = 0.3436

In [31]:
# Treinar árvore de decisão com o melhor alpha
modelo_arvore_decisao_alpha = DecisionTreeRegressor(random_state=42,
                                                    max_depth=5,
                                                    min_samples_split=10, 
                                                    min_samples_leaf=5,
                                                    splitter='best',
                                                    # ccp_alpha=melhor_mae['ccp_alpha'])
                                                    ccp_alpha=ccp_alpha)

modelo_arvore_decisao_alpha.fit(X_train, y_train['NUM_NOTA_CH'])

DecisionTreeRegressor(ccp_alpha=0.3436, max_depth=5, min_samples_leaf=5,
                      min_samples_split=10, random_state=42)

In [32]:
# Avaliação do modelo
y_pred_arvore_decisao_alpha = modelo_arvore_decisao_alpha.predict(X_test)

In [33]:
# Iniciar o rastreamento do MLflow
with mlflow.start_run() as run:

    # Registrar os parâmetros
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("min_samples_split", 10)
    mlflow.log_param("min_samples_leaf", 5)
    mlflow.log_param("splitter", "best")
    mlflow.log_param("random_state", 42)
    mlflow.log_param("ccp_alpha", ccp_alpha)

    # Registrar as métricas
    r2_arvore_decisao_alpha = r2_score(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_alpha)
    mae_arvore_decisao_alpha = mean_absolute_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_alpha)
    rmse_arvore_decisao_alpha = root_mean_squared_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_alpha)

    mlflow.log_metric("r2", r2_arvore_decisao_alpha)
    mlflow.log_metric("mae", mae_arvore_decisao_alpha)
    mlflow.log_metric("rmse", rmse_arvore_decisao_alpha)

    # Definir uma TAG para o modelo
    mlflow.set_tag("model_type", "Decision Tree Regressor com alpha otimizado")

    # Inferir assinatura do modelo
    signature = mlflow.models.infer_signature(X_train, y_train['NUM_NOTA_CH'])

    # Registrar modelo
    mlflow.sklearn.log_model(sk_model=modelo_arvore_decisao_alpha,
                             artifact_path="modelo_arvore_decisao_alpha",
                             signature=signature,
                             registered_model_name="modelo_arvore_decisao_alpha")


Registered model 'modelo_arvore_decisao_alpha' already exists. Creating a new version of this model...
2025/05/02 10:47:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_arvore_decisao_alpha, version 5


🏃 View run big-rook-716 at: http://127.0.0.1:9080/#/experiments/299918284299748162/runs/94e4b383afc341e0b1aa6e9236d5c7d8
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/299918284299748162


Created version '5' of model 'modelo_arvore_decisao_alpha'.


In [34]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_arvore_decisao_alpha.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_alpha, "teste")

MAE (treino): 57.71
RMSE (treino): 72.78
R2 (treino): 0.26
MAE (teste): 57.73
RMSE (teste): 72.85
R2 (teste): 0.26


In [ ]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_arvore_decisao_alpha, 'Modelos\modelo_arvore_decisao_alpha.pkl')

['Modelos\\modelo_arvore_decisao_alpha.pkl']

Grid Search

In [37]:
# Configurar os parâmetros para o GridSearchCV

param_grid = {
    'ccp_alpha': [0.2, 0.3436, 0.4],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'splitter': ['best', 'random']
}

In [38]:
# Validação cruzada para regressão
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Criar o modelo de árvore de decisão
modelo_arvore_decisao_grid = DecisionTreeRegressor(random_state=42)

In [39]:
# Realizar o GridSearchCV
grid_search = GridSearchCV(estimator=modelo_arvore_decisao_grid,
                           param_grid=param_grid,
                           scoring='r2',
                           cv=kf,
                           return_train_score=True,
                           n_jobs=-1,
                           verbose=1)

In [40]:
# Executar o GridSearchCV
grid_search.fit(X_train, y_train['NUM_NOTA_CH'])

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'ccp_alpha': [0.2, 0.3436, 0.4],
                         'max_depth': [3, 5, 7, 10],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             return_train_score=True, scoring='r2', verbose=1)

In [41]:
# Melhores parâmetros encontrados
melhores_parametros = grid_search.best_params_
print(f"Melhores parâmetros: {melhores_parametros}")

Melhores parâmetros: {'ccp_alpha': 0.2, 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2, 'splitter': 'best'}


In [42]:
# Treinar o modelo com os melhores parâmetros
modelo_arvore_decisao_best = DecisionTreeRegressor(random_state=42, **melhores_parametros)
modelo_arvore_decisao_best.fit(X_train, y_train['NUM_NOTA_CH'])

DecisionTreeRegressor(ccp_alpha=0.2, max_depth=10, min_samples_leaf=5,
                      random_state=42)

In [43]:
# Avaliação do modelo final
y_pred_arvore_decisao_best = modelo_arvore_decisao_best.predict(X_test)

In [44]:
# Iniciar o rastreamento do MLflow - modelo com GridSearchCV

with mlflow.start_run() as run:

    # Registrar os parâmetros
    for param, value in melhores_parametros.items():
        mlflow.log_param(param, value)

    # Registrar as métricas
    r2_arvore_decisao_best = r2_score(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_best)
    mae_arvore_decisao_best = mean_absolute_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_best)
    rmse_arvore_decisao_best = root_mean_squared_error(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_best)

    mlflow.log_metric("r2", r2_arvore_decisao_best)
    mlflow.log_metric("mae", mae_arvore_decisao_best)
    mlflow.log_metric("rmse", rmse_arvore_decisao_best)

    # Definir uma TAG para o modelo
    mlflow.set_tag("model_type", "Decision Tree Regressor com GridSearchCV")

    # Inferir assinatura do modelo
    signature = mlflow.models.infer_signature(X_train, y_train['NUM_NOTA_CH'])

    # Registrar modelo
    mlflow.sklearn.log_model(sk_model=modelo_arvore_decisao_best,
                             artifact_path="modelo_arvore_decisao_grid",
                             signature=signature,
                             registered_model_name="modelo_arvore_decisao_grid") 

Registered model 'modelo_arvore_decisao_grid' already exists. Creating a new version of this model...
2025/05/02 11:10:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_arvore_decisao_grid, version 3


🏃 View run treasured-dove-153 at: http://127.0.0.1:9080/#/experiments/299918284299748162/runs/8933c35177644907974b2c35383c4100
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/299918284299748162


Created version '3' of model 'modelo_arvore_decisao_grid'.


In [45]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_arvore_decisao_best.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_arvore_decisao_best, "teste")

MAE (treino): 56.53
RMSE (treino): 71.49
R2 (treino): 0.29
MAE (teste): 56.73
RMSE (teste): 71.76
R2 (teste): 0.28


In [ ]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_arvore_decisao_best, 'Modelos\modelo_arvore_decisao_best.pkl')

['Modelos\\modelo_arvore_decisao_best.pkl']